In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
#rawdata = trainendpts[0]
rawdata = np.transpose(np.delete(np.genfromtxt('VP16longdata50.csv',delimiter=','),0,0))
#print(len(rawdata))
#print(rawdata)
newdata = list(rawdata.flatten())
#print(newdata)
parameters = (np.delete(np.genfromtxt('senseParams.csv', delimiter=','),0,0))
lightdata = np.transpose(np.delete(np.genfromtxt('longlight50.csv', delimiter=','),0,0))
#print(len(lightdata))
#print(len(lightdata))

print(len(parameters))
t = np.linspace(0,61200, num=12241)
#t = np.linspace(0,34800, num=6961)

#print(t)
 
def func(t):
    z0 = [1,0,0,0,0,0,0]
    
    arrayvalues = np.asarray([])
    end = np.zeros((len(parameters), len(lightdata[:,0]), len(t)))
    print(len(end))
    
    for i in range(2):
        def I(t):
            tindex = t/5
            if tindex > 12241:
                tindex = 12240
            return lightdata[i][int(tindex)]
  
        for j in range(len(parameters)):
        #for j in range(2):
                def model(z,t):
                    p1 = parameters[:,0];
                    p2 = parameters[:,1];
                    p3 = parameters[:,2];
                    #p4 = parameters[:,3];

                    d2 = p1[j]
                    k3 = p3[j]
                    k2 = p2[j]
                    #k7 = p4[j]
                    
                    Kd = 90.41
                    n = 0.964487
                    #d2 =  22.04
                    #k2 = 0.00077
                    #k3 = 0.0019
                    #k7 = 8.84
                    d1 = 0.019905
                    k1 = 0.08299
                    d3 = 0.000077
                    k4 = 1.25
                    d4 = 0.000031
                    k5 = 0.00283
                    k6 = 0.00283
                    
                    Pu = z[0]
                    Pb = z[1]
                    Pa = z[2]
                    mRNA = z[3]
                    mCherry1 = z[4]
                    mCherry2 = z[5]
                    mCherry3 = z[6]


                    dPudt = d1*Pb - k1*I(t)**n/(Kd**n+I(t)**n)*Pu 
                    dPbdt = k1*I(t)**n/(Kd**n+I(t)**n)*Pu - k2*Pb - d1*Pb + d2*Pa
                    dPadt = k2*Pb - d2*Pa 
                    dmRNAdt = k3*Pa - d3*mRNA
                    dmCherry1dt = k4*mRNA-(d4 + k5)*mCherry1
                    dmCherry2dt = k5*mCherry1-(d4+k6)*mCherry2
                    dmCherry3dt = k6*mCherry2 - d4*mCherry3
 
                    return [dPudt,dPbdt,dPadt,dmRNAdt,dmCherry1dt,dmCherry2dt,dmCherry3dt]  


                z = odeint(model,z0,t,hmax=1)
                mCherry3 = z[:,6]
                end[j,i] = mCherry3

    return end


In [ ]:
modelout = np.asarray(func(t))

model1 = np.asarray(modelout)
#model1 = modelout
ydata = np.asarray(rawdata)
ydata=list(ydata.flatten())

#ydata=np.asarray(ydata)
#print('end = ', model1);

print(len(ydata))



In [ ]:

R2all = np.zeros((len(parameters)))

for j in range(len(parameters)):
    #print(ydata)
    print((modelout[j,:]))
    model1 = modelout[j,:]
    mCherry3=list(model1.flatten())
    mCherry3= np.asarray(mCherry3)
    ssr = np.sum((ydata-mCherry3)**2)
    #print(ssr)
    sst = np.sum((ydata - np.mean(ydata))**2)
    #print(sst)
    R2 = 1 - ssr/sst
    #print('R2_'+str(j+1), '=',R2)

    R2all[j]=R2
                  
print(R2all)

In [ ]:
import csv 
pp = PdfPages('multipage.pdf')

for i in range(len(parameters)): 
    with open('20250212_VP16_3st1hillA_smaller_50_all_R2_10k.csv','w') as csvfile: 
        csvwriter=csv.writer(csvfile)

        csvwriter.writerow(R2all)

#with open('20250112_VP16_all_3st1hillA_smaller_50_modelout_10k.csv','w') as csvfile:
    #csvwriter=csv.writer(csvfile)

    #csvwriter.writerow(modelout)

In [ ]:
data = (np.delete(np.genfromtxt('20250210_VP16_FBLC_smaller_50_all_R2_10k.csv', delimiter=','),0,0))
parameters = (np.delete(np.genfromtxt('senseParams.csv', delimiter=','),0,0))
#R2all = data[:,0]
#n = data[:,1]
#n = data[:,2]
#n = data[:,3]
R2all = data
print(len(R2all))
#print(len(d2))
d2 = parameters
print(len(n))
pp = PdfPages('multipage.pdf')

#print(parameters)

#plot
plt.figure(1)
plt.scatter(d2, R2all, c='gray')
plt.plot(d2[R2all > 0.8], R2all[R2all > 0.8], 'co')
max_index = np.argmax(R2all)
plt.plot(d2[max_index], R2all[max_index], 'ko')
plt.xlabel('d2 [s$^-$$^1$]')
plt.ylabel('R$^2$')
plt.xscale('log')
plt.ylim((0,1))
plt. suptitle('VP16 FBLC ',fontsize = 14)
plt.savefig("VP16_d2_FBLC_smaller_50_10k.svg")
plt.show


#cmap=plt.get_cmap('magma')
#min_cmap= 0.015
#max_cmap=0.02#

#plt.figure(2)
#for i in range(len(lightdata)):
    #for j in range(len(parameters)):
        #print(ydata)
        #print((modelout[j,:]))
        #model=modelout[j,i]
        #plt.plot(t,model)

        




#plt.figure(2)
###n plot
#plt.scatter(n, R2all, c='gray')
#plt.plot(n[R2all > 0.9], R2all[R2all > 0.9], 'co')
#max_index = np.argmax(R2all)
#plt.plot(n[max_index], R2all[max_index], 'ko')
##plt.yscale('log')
#plt.xscale('log')

#plt.xlabel('n [s$^-$$^1$]')
#plt.ylabel('R$^2$')
#plt.ylim((0,1))

#plt. suptitle('VP16 3st1hillA ',fontsize = 14)
#plt.savefig("VP16_n_all_3st1hillA.svg")
#plt.show

#plt.figure(3)
##n plot
#plt.scatter(n, R2all, c='gray')
#plt.plot(n[R2all > 0.9], R2all[R2all > 0.9], 'co')
#max_index = np.argmax(R2all)
#plt.plot(n[max_index], R2all[max_index], 'ko')
#plt.xlabel('n [s$^-$$^1$]')
#plt.ylabel('R$^2$')
#plt.xscale('log')
#plt.ylim((0,1))

#plt. suptitle('VP16 3st1hillA ',fontsize = 14)
#plt.savefig("VP16_n_all_3st1hillA.svg")
#plt.show
#for j in range(len(parameters)):
#    

#    plt.plot(parameters[j],R2,'*')
#    plt. suptitle('parameter combo '+str(j+1),fontsize = 14)
#    pp.savefig()
#    plt.show()
#pp.close()

In [ ]:
import colormaps as cmaps

from colormaps.utils import show_cmaps_collection



data = (np.delete(np.genfromtxt('20250212_VP16_3st1hillA_smaller_50_all_R2_10k_good.csv', delimiter=','),0,0))
#parameters = (np.delete(np.genfromtxt('senseParams.csv', delimiter=','),0,0))
print(len(data))
#print(len(parameters))

d2 = data[:,1]
k2 = data[:,2]
k3 = data[:,3]
#k7 = data[:,4]
R2all = data[:,0]

#plt.figure(1)
#hb = plt.hexbin(k7,k2, C=R2all, gridsize=18, cmap=cmaps.freeze_r, edgecolors='k', xscale='log', yscale='log',vmin=0,vmax=1)
##plt.scatter(k7,k2, s=2, c="white")
#plt.colorbar(hb)
#plt.xlabel('k7 [s$^-$$^1$]')
#plt.ylabel('k2 [s$^-$$^1$]')
#plt.savefig("VP16_k7_k2_hexbin_3st1hill_20k.svg")

#plt.figure(2)
#hb = plt.hexbin(k7, d2, C=R2all, gridsize=18, cmap=cmaps.freeze_r, edgecolors='k', xscale='log', yscale='log',vmin=0,vmax=1)
##plt.scatter(k7,d2, s=2, c="white")
#plt.xlabel('k7 [s$^-$$^1$]')
#plt.ylabel('d2 [s$^-$$^1$]')
#plt.colorbar(hb)
#plt.savefig("VP16_k7_d2_hexbin_3st1hill_20k.svg")

#plt.figure(3)
#hb = plt.hexbin(k7, k3, C=R2all, gridsize=18, cmap=cmaps.freeze_r, edgecolors='k', xscale='log',yscale='log',vmin=0,vmax=1)
##plt.scatter(k7,k3, s=2, c="white")
#plt.colorbar(hb)
#plt.xlabel('k7 [s$^-$$^1$]')
#plt.ylabel('k3 [s$^-$$^1$]')
#plt.savefig("VP16_k7_k3_hexbin_3st1hill_20k.svg",bbox_inches='tight')

plt.figure(4)
hb = plt.hexbin(k2, k3, C=R2all, gridsize=18, cmap=cmaps.freeze_r, edgecolors='k',xscale='log', yscale='log',vmin=0,vmax=1)
#plt.scatter(k2,k3, s=2, c="white")
plt.colorbar(hb)
plt.xlabel('k2 [s$^-$$^1$]')
plt.ylabel('k3 [s$^-$$^1$]')
plt.savefig("VP16_k2_k3_hexbin_3st1hill_50_20k.svg",bbox_inches='tight')

plt.figure(5)
hb = plt.hexbin(d2, k3, C=R2all, gridsize=18, cmap=cmaps.freeze_r, edgecolors='k',xscale='log', yscale='log',vmin=0,vmax=1)
#plt.scatter(d2,k3, s=2, c="white")
plt.colorbar(hb)
plt.xlabel('d2 [s$^-$$^1$]')
plt.ylabel('k3 [s$^-$$^1$]')
plt.savefig("VP16_d2_k3_hexbin_3st1hill_50_20k.svg",bbox_inches='tight')


plt.figure(6)
hb = plt.hexbin(k2, d2, C=R2all, gridsize=18, cmap=cmaps.freeze_r, edgecolors='k', xscale='log', yscale='log',vmin=0,vmax=1)
#plt.scatter(k2,d2, s=2, c="white")
plt.colorbar(hb)
plt.xlabel('k2 [s$^-$$^1$]')
plt.ylabel('d2 [s$^-$$^1$]')
plt.savefig("VP16_k2_d2_hexbin_3st1hill_50_20k.svg",bbox_inches='tight')



In [ ]:
pip install colormaps
